In [71]:
"""
Queries Uniprot database and retrieves protein data.
"""
import urllib.request
from urllib.error import HTTPError
from lxml import etree
import lxml.etree
import lxml.html
import json
import xmlschema
import lxml
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd

def unroll(data, master_list, ec):
    if type(data) == dict:
        for key, value in data.items():
            if(key == 'ecNumber'):
                ec = True
            unroll(data[key], master_list, ec)  
    elif type(data) == list:
        for i in range(0,len(data)):
            unroll(data[i], master_list, ec)
    else:
        if(type(data) != int):
            if(ec == True):
                data = "EC " + data
            master_list.append(data)

def findNewIDforOldID(oldID):
    A = 0
    print('Looking for the oldID', oldID)
    source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
    webpage = BeautifulSoup(source, 'html')
    main = webpage.find('main')
    altIDsSection = main.find_all('div', class_= 'entryEmpty')
    altIDs = []
    for each_ID in altIDsSection:
        links = each_ID.find_all('a', href=True)
        for link in links:
            altIDlink = link['href'][9:]   #removes prefix of link containing ID
            altIDs.append(altIDlink)
    
    #Remove non-protein IDs from altIDs
    endOfaltIDs = 0
    for index in range(0, len(altIDs)):
        if('replaces' in altIDs[index]):
            endOfaltIDs = index
            break
    realAltIDs = []
    for index in range(0, endOfaltIDs):
        realAltIDs.append(altIDs[index])
    print('Stopped looking for the old ID. Found ', realAltIDs)
    return(realAltIDs)

def findNewIDforOldID_SecondAttempt(oldID):
    #FIRST: Finding the link to the next page /html/body/main/div/p/a[1]
    source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
    webpage = BeautifulSoup(source, 'html')
    altID_Link = 'https://www.uniprot.org'+webpage.find('main').find('div', class_= 'entryEmpty').find("p").find("a")['href']
    
    print('The protein ID will be looked for at this link: ', altID_Link)

    #SECOND: Using the link for the next page to find the existing ID: /html/body/main/div/div[3]/table/tbody/tr/td[4]/a[2
                                                                    #  /html/body/main/div/div[3]/div[4]/table/tbody/tr[1]/td[2]/div/div]
    source2 = requests.get(altID_Link).text
    webpage2 = BeautifulSoup(source2, 'html')
    main = webpage2.find('main',class_='uniparc') #.find('div', class_='content results').find('table', class_='grid')#.find('tbody')
    step1 = main.find('div',class_='main').find('div',class_='content results').find('table').find('tbody').find('tr').find_all('td')
    thePossibleIDs = step1[3].find_all('a')
    thePossibleIDs_List = []
    for ID_Link in thePossibleIDs:
        thePossibleIDs_List.append(ID_Link['href'][9:])
    #Finds the ID(s) out of the final options from the last section
    theRightIDs = []
    for index in range(0, len(thePossibleIDs_List)):
        if('version' not in thePossibleIDs_List[index]):
            theRightIDs.append(thePossibleIDs_List[index])
    return(theRightIDs)

def findURLxml(ID):
    unrolledSynList = []
    url = 'https://www.uniprot.org/uniprot/'+ID+'.xml'
    with urllib.request.urlopen(url) as r:                     
        raw_data = r.read().strip()
    tree = etree.fromstring(raw_data); 
    entry_dict = schema.to_dict(tree);
    content = entry_dict['entry'][0]
    try: species_name = content['organism']['name'][0]['$']
    except: species_name = ''; print('Error with species name for ', ID)
    unroll(content['protein'], unrolledSynList, False)
    return unrolledSynList, species_name

def findUniRefURL(ID):
    webpage = BeautifulSoup(requests.get('https://www.uniprot.org/uniref/?query='+ID+'&fil=identity:1.0&sort=score').text, 'html')
    try: syn = str(webpage.find('main', class_= 'uniref').find('div', class_="main-aside").find('div', class_="content results").find('table', class_="grid").find('tbody').find('tr').find_all('td')[3]).split('>')[1].split(': ')[1].split('<')[0]    
    except: syn = []
    try: species_name = str(webpage.find('main', class_='uniref').find('div', class_="main-aside").find('div', class_="content results").find('table').find('tbody').find('tr').find_all('td')[6]).split('>')[2].split('<')[0]     #/html/body/main/div/div[2]/table/tbody/tr/td[7]/a
    except: species_name = ''
    #print(species_name)
    #print(syn)
    return syn, species_name


def findURL(ID):
    synList = []; species_name = 'N/A'; stillMissIDs = []; foundidList = []; notfoundidList = []
    try: 
        synList, species_name = findURLxml(ID); print('1st Try')
        foundidList.append(ID)
    except: 
        print('Not found on 1st try')
        try: 
            missing = False; UniRef = False
            altIDs = findNewIDforOldID(ID) #To find synonyms. If found on 2nd try. There might be multiple new IDs
            # If this is a dead end and no alternate IDs are provided, try UniRef
            if(altIDs == [] or altIDs == '[]'):
                try: 
                    SYN, SN = findUniRefURL(ID)
                    SYN = SYN.split(' ')[0]
                    if(SYN != [] and SYN != '[]'): print('SYN found on UniRef ', SYN, 'in ', SN); UniRef = True; altIDs = [ID]
                except: 
                    print('Couldn\'t find on UniRef either')
            # IF UniProt provides alternates, use those IDs to get their synonyms/names
            for altID in altIDs:
                if(UniRef == False):
                    try: syns, sn = findURLxml(altID); 
                    except: missing = True               # ...unless 
                elif(UniRef == True):
                    syns, species_name = [SYN], SN
                for s in syns: synList.append(s)
                foundidList.append(altID)
                print('2nd try')
                print('New ID for ', ID, ' is ', altID, ' on 2nd try')
                if missing == True: stillMissIDs.append(altID)
        
        except: print('Not all found on 2nd try'); stillMissIDs.append(ID) #If there weren't any new IDs
        if(len(stillMissIDs) > 0): #If there are still IDs without an entry
            for ID in stillMissIDs:
                try: 
                    altIDs2 = findNewIDforOldID_SecondAttempt(ID); print('3rd try')
                    for altID2 in altIDs2:
                        try: 
                            syns, sn = findURLxml(altID2) 
                            for s in syns: synList.append(s)
                            foundidList.append(altID2)
                            print('New ID for ', ID, ' is ', altID, ' on 3rd try')
                        except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(altID2)
                except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(ID)
    for missingname in notfoundidList: print('No Entry Found For ', missingname)
    return notfoundidList, foundidList, synList, species_name

In [48]:
df_scores = pd.read_csv(r'./../caseolap/result/caseolap.csv')
df_scores.rename(columns={'protein':'ID'}, inplace = True)
df_scores

,ID,LHON,MELAS,ADOA,Leigh,MERRF,KSS,Alpers,LBSL,PDD,PEO,MS,Parkin,ALS,AD,Epil,Schiz,FA,HSP,WD
0,Q96DA6,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005083,0.000000
1,Q96IX5,0.0,0.0,0.00000,0.014041,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.003432,0.000000,0.0,0.0,0.000000,0.000000
2,Q92947,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.007768,0.040287,0.0,0.0,0.000000,0.000000
3,Q9P2J9,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.003371,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,Q9H4I9,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005083,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,Q96KJ9,0.0,0.0,0.00496,0.013259,0.0,0.0,0.000000,0.0,0.0,0.000000,0.003835,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
724,Q32NB8,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.003435,0.000000,0.0,0.0,0.000000,0.000000
725,Q9UGC7,0.0,0.0,0.00000,0.004371,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
726,Q16854,0.0,0.0,0.00000,0.000000,0.0,0.0,0.004933,0.0,0.0,0.009951,0.000000,0.000000,0.0,0.003405,0.003557,0.0,0.0,0.000000,0.021897


In [49]:
f = open('./../MitoProteins.txt', 'r')
MitoProteins = {}
for line in f:
    ID = line.split('|')[0]
    names = line.split('|')[1:]
    MitoProteins[ID] = list(names)

In [50]:
name_col = []
other_name_col = []
print('Row Count:', len(df_scores))
for i in range(0,len(df_scores)):
    protein_name = MitoProteins[df_scores.iloc[i,0]][0]
    other_names = MitoProteins[df_scores.iloc[i,0]][1:]
    name_col.append(protein_name)
    other_name_col.append(other_names)

Row Count: 728


In [51]:
df_scores = df_scores.assign(name = name_col, synonyms = other_name_col)
df_scores

,ID,LHON,MELAS,ADOA,Leigh,MERRF,KSS,Alpers,LBSL,PDD,...,Parkin,ALS,AD,Epil,Schiz,FA,HSP,WD,name,synonyms
0,Q96DA6,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005083,0.000000,mitochondrial import inner membrane translocas...,"[DNAJC19 isoform 5, PAM18, DNAJC19 isoform 1, ..."
1,Q96IX5,0.0,0.0,0.00000,0.014041,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.003432,0.000000,0.0,0.0,0.000000,0.000000,up-regulated during skeletal muscle growth pro...,"[ATP synthase membrane subunit DAPIT, mitochon..."
2,Q92947,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.007768,0.040287,0.0,0.0,0.000000,0.000000,"glutaryl-CoA dehydrogenase, mitochondrial isof...","[glutaryl-CoA dehydrogenase, mitochondrial, Tr..."
3,Q9P2J9,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.003371,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,PDP 2,"[PDP2, Pyruvate dehydrogenase phosphatase cata..."
4,Q9H4I9,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005083,0.000000,"Essential MCU regulator, mitochondrial",[single-pass membrane protein with aspartate r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,Q96KJ9,0.0,0.0,0.00496,0.013259,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,dJ857M17.2,"[COXIV-2, cytochrome c oxidase subunit 4 isofo..."
724,Q32NB8,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.003435,0.000000,0.0,0.0,0.000000,0.000000,CDP-diacylglycerol--glycerol-3-phosphate 3-pho...,[CDP-diacylglycerol--glycerol-3-phosphate 3-ph...
725,Q9UGC7,0.0,0.0,0.00000,0.004371,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,MTRF1L,"[Peptide chain release factor 1-like, mitochon..."
726,Q16854,0.0,0.0,0.00000,0.000000,0.0,0.0,0.004933,0.0,0.0,...,0.000000,0.0,0.003405,0.003557,0.0,0.0,0.000000,0.021897,DGUOK isoform 2,"[Deoxyguanosine kinase, mitochondrial isoform ..."


## Adding 'Interactor' Proteins
These are proteins which Reactome said interacts with the submitted protein list. 
These proteins will be added to the database as proteins, although without CaseOLAP scores

In [52]:
df_reactome = pd.read_csv('Mitochondrial Proteome 1135.csv')
df_reactome

,Pathway identifier,Pathway name,#Entities found,#Entities total,#Interactors found,#Interactors total,Entities ratio,Entities pValue,Entities FDR,#Reactions found,#Reactions total,Reactions ratio,Species identifier,Species name,Submitted entities found,Mapped entities,Submitted entities hit interactor,Interacts with,Found reaction identifiers
0,R-HSA-1268020,Mitochondrial protein import,65,69,2,15,0.004745,1.110000e-16,1.840000e-14,14,14,0.001036,9606,Homo sapiens,O75746;Q96H96;Q9Y584;P56277;Q9BVV7;O96008;Q9NX...,O75746;Q96H96;Q9Y584;P56277;Q9BVV7;O96008;Q9NX...,Q9Y5J7;P62072,Q9Y5J6,R-HSA-1299475;R-HSA-1268022;R-HSA-1299478;R-HS...
1,R-HSA-5389840,Mitochondrial translation elongation,87,94,2,43,0.006464,1.110000e-16,1.840000e-14,8,8,0.000592,9606,Homo sapiens,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q8IVH4;P22033,15996,R-HSA-5389842;R-HSA-5389857;R-HSA-5419268;R-HS...
2,R-HSA-5419276,Mitochondrial translation termination,87,94,2,45,0.006464,1.110000e-16,1.840000e-14,5,5,0.000370,9606,Homo sapiens,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q96A35;Q96DV4;Q96EL2;Q7Z2W9;Q96EL3;Q9P015;Q9H0...,Q8IVH4;P22033,15996,R-HSA-5419264;R-HSA-5419281;R-HSA-5419271;R-HS...
3,R-HSA-5368287,Mitochondrial translation,93,102,8,132,0.007014,1.110000e-16,1.840000e-14,17,17,0.001258,9606,Homo sapiens,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q9BVS5;Q8IVH4;Q8NI60;P22033;Q07817;Q9BUP3;O148...,Q9H2K0;P46199;15996,R-HSA-5389842;R-HSA-5419281;R-HSA-5389841;R-HS...
4,R-HSA-5368286,Mitochondrial translation initiation,87,96,6,86,0.006602,1.110000e-16,1.840000e-14,4,4,0.000296,9606,Homo sapiens,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q96A35;Q96DV4;P46199;Q96EL2;Q7Z2W9;Q96EL3;Q9P0...,Q9BVS5;Q8NI60;Q07817;Q9BUP3;O14880;P17152,Q9H2K0;P46199,R-HSA-5389841;R-HSA-5368279;R-HSA-5389849;R-HS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,R-HSA-9635486,Infection with Mycobacterium tuberculosis,0,134,30,724,0.009215,1.000000e+00,1.000000e+00,8,73,0.005401,9606,Homo sapiens,NaN,NaN,Q9BSH4;Q9BXW6;Q969S9;Q08426;Q9H3L0;Q96E11;Q4G1...,P27361;P28482;16856;P49840;P51149;O14964;P1437...,R-HSA-9635667;R-HSA-9636304;R-HSA-9635424;R-HS...
1581,R-HSA-8986944,Transcriptional Regulation by MECP2,1,100,68,1162,0.006877,1.000000e+00,1.000000e+00,8,77,0.005697,9606,Homo sapiens,P17612,P17612,Q969Q5;Q02218;Q16891;P45880;Q96EY1;Q9NPL8;Q6DK...,Q05329;P60484;P08581;P42858;P18031;Q14353,R-HSA-8986937;R-HSA-9023592;R-HSA-9615571;R-HS...
1582,R-HSA-1236394,Signaling by ERBB4,0,82,14,636,0.005639,1.000000e+00,1.000000e+00,5,52,0.003847,9606,Homo sapiens,NaN,NaN,Q9Y3D2;P17612;Q9BYZ2;Q8IVH4;Q9H7Z7;Q92843;Q96B...,P14136;16618;P04271;P02649;15996,R-HSA-9612243;R-HSA-1250370;R-HSA-1253321;R-HS...
1583,R-HSA-157118,Signaling by NOTCH,0,258,30,751,0.017742,1.000000e+00,1.000000e+00,14,154,0.011394,9606,Homo sapiens,NaN,NaN,P49406;P49327;P49916;P57105;Q5T1C6;Q16822;O603...,P41743;P05412;P31749;O00308;P84022;P01106;P678...,R-HSA-9021357;R-HSA-2168136;R-HSA-9017797;R-HS...


In [53]:
num = []
pathway_id = []
pathway_name = []
interactor_row = []
interacts_row = []
uniprotless_pathway = []
uniprot_dict = {}
interactor_dict = {}
interact_dict = {}
x = 0
for i in range(0,len(df_reactome)):
    if(type(df_reactome['Submitted entities hit interactor'][i]) == str):
        interactor = df_reactome['Submitted entities hit interactor'][i].split(';')
        interacts = df_reactome['Interacts with'][i].split(';')
        num.append(x); x += 1
        for b in range(0,len(interacts)):
            for j in range(0,len(interactor)):
                interacts_row.append(interacts[b])
                interactor_row.append(interactor[j])
        interactor_dict[i] = interactor_row
        interact_dict[i] = interacts_row
    else:
        uniprotless_pathway.append(df_reactome['Submitted entities hit interactor'][i])
        #print(i, ': ', df_reactome['Pathway identifier'][i])
print('Interactorless Pathway: ', len(uniprotless_pathway), '/', len(df_reactome), '=', len(uniprotless_pathway)/len(df_reactome))
#df_reactome2 = pd.DataFrame({'num': num, 'Submitted entities hit interactor':interactor_dict, 'Interacts with':interacts_dict})
#df_reactome2

Interactorless Pathway:  171 / 1585 = 0.10788643533123028


In [54]:
A_HIC = []
B_IWC = []
C_n = []
n = 0
for i in range(0,len(df_reactome)):
    hit_interactor_cell = df_reactome['Submitted entities hit interactor'][i]
    interacts_with_cell = df_reactome['Interacts with'][i]
    if(type(hit_interactor_cell) == str and type(interacts_with_cell) == str):
        hit_interactor_cell = hit_interactor_cell.split(';')
        interacts_with_cell = interacts_with_cell.split(';')
        for j in range(0,len(interacts_with_cell)):
            for k in range(0,len(hit_interactor_cell)):       
                A_HIC.append(hit_interactor_cell[k])
                B_IWC.append(interacts_with_cell[j])
                C_n.append(n)
                n += 1
df_interact = pd.DataFrame({'#':C_n,'Submitted Protein':A_HIC,'Interacts With':B_IWC})

In [56]:
IUP_set = {df_interact['Interacts With'][0]}
for i in range(0,len(df_interact)):
    IUP_set.add(df_interact['Interacts With'][i])
interact_with = list(IUP_set)
print('Unique Interactor Proteins:', len(IUP_set))

Unique Interactor Proteins: 731


In [73]:
format = '.xml'; schema = xmlschema.XMLSchema('https://www.uniprot.org/docs/uniprot.xsd')
InteractorDict = {}; initial = 0;i = initial
synList = []; notfound = []; notfoundDict = {}

for ID in interact_with[initial:]:
    if(ID in MitoProteins.keys()):
        print("Interactor is a submitted proteins")
    else:
        notfoundIDs, foundIDs, synList, sn = findURL(ID)
    InteractorDict[ID] = synList

1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
Interactor is a submitted proteins
Interactor is a submitted proteins
1st Try
Interactor is a submitted proteins
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
Interactor is a submitted proteins
Interactor is a submitted proteins
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
Not found on 1st try
Looking for the oldID 15996
Stopped looking for the old ID. Found  []
SYN found on UniRef  Protein in  Caenorhabditis briggsae
2nd try
New ID for  15996  is  15996  on 2nd try
1st Try
1st Try
Interactor is a submitted proteins
Interactor is a submitted prote

1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
Not found on 1st try
Looking for the oldID 28494
Stopped looking for the old ID. Found  []
SYN found on UniRef  dimethylarginine in  Rathayibacter iranicus
2nd try
New ID for  28494  is  28494  on 2nd try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Not found on 1st try
Looking for the oldID 17526
Stopped looking for the old ID. Found  []
SYN found on UniRef  hypothetical in  Streptomyces sp. PKU-EA00015
2nd try
New ID for  17526  is  17526  on 2nd try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
Interactor is a submitted proteins
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1st Try
1s

In [79]:
for ID,names in InteractorDict.items():
    df_scores = df_scores.append({'ID':ID, 'name':names[0], 'synonyms':names[1:]}, ignore_index = True)

for ID,names in MitoProteins.items():
    if(ID not in list(df_scores['ID'])):
        df_scores = df_scores.append({'ID':ID, 'name':names[0], 'synonyms':names[1:]}, ignore_index = True)

In [85]:
# Adding Mitochondrial proteins which didn't have CaseOLAP scores Not really using them, but they'll be in the database just because. 
for ID,names in MitoProteins.items():
    if(ID not in list(df_scores['ID'])):
        df_scores = df_scores.append({'ID':ID, 'name':names[0], 'synonyms':names[1:]}, ignore_index = True)

In [86]:
df_scores

,ID,LHON,MELAS,ADOA,Leigh,MERRF,KSS,Alpers,LBSL,PDD,...,Parkin,ALS,AD,Epil,Schiz,FA,HSP,WD,name,synonyms
0,Q96DA6,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005083,0.0,mitochondrial import inner membrane translocas...,"[DNAJC19 isoform 5, PAM18, DNAJC19 isoform 1, ..."
1,Q96IX5,0.0,0.0,0.0,0.014041,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.003432,0.000000,0.0,0.0,0.000000,0.0,up-regulated during skeletal muscle growth pro...,"[ATP synthase membrane subunit DAPIT, mitochon..."
2,Q92947,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.007768,0.040287,0.0,0.0,0.000000,0.0,"glutaryl-CoA dehydrogenase, mitochondrial isof...","[glutaryl-CoA dehydrogenase, mitochondrial, Tr..."
3,Q9P2J9,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.003371,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,PDP 2,"[PDP2, Pyruvate dehydrogenase phosphatase cata..."
4,Q9H4I9,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.005083,0.0,"Essential MCU regulator, mitochondrial",[single-pass membrane protein with aspartate r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,A0A0B4J2F0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PIGBOS1 isoform 1,"[HP06981, PIGB opposite strand protein 1, Prot..."
1846,P86397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTD2 isoform 7,"[HTD2 isoform 3, 3-hydroxyacyl-thioester dehyd..."
1847,PROT_1135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RP11_469A15.2\n,[]
1848,A0A0B4J2D5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Keio novel protein-I,[Glutamine amidotransferase-like class 1 domai...


In [87]:
df_scores.to_csv('data/protein-nodes.csv')